In [1]:
import pandas as pd
import numpy as np
import uproot
import matplotlib
from matplotlib import pyplot as plt
from __future__ import division
from tqdm import tqdm

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

normalize = False

# Get Files

In [2]:
input_bkg = [#'trees/MaxDigis_tree_Neutrino_gun.root',
             'trees/MaxDigis_tree_Data.root',
            ]
first_sig = ['trees/MaxDigis_tree_ppTohToSSTo4b.root']
#first_sig = ['trees/MaxDigis_tree_HTo2LongLivedTo4q_MH_125_MFF_1_CTau_5000mm_TuneCP5_14TeV_pythia.root']
input_sig = ['trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-25_CTau-15000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-25_CTau-1500mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-12_CTau-9000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-12_CTau-900mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-50_CTau-30000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-50_CTau-3000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-250_MFF-120_CTau-10000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-250_MFF-120_CTau-1000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-250_MFF-120_CTau-500mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-250_MFF-60_CTau-10000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-250_MFF-60_CTau-500mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-350_MFF-160_CTau-10000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-350_MFF-160_CTau-1000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-350_MFF-160_CTau-500mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-350_MFF-80_CTau-10000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-350_MFF-80_CTau-1000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-350_MFF-80_CTau-500mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-1000_MFF-450_CTau-100000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-1000_MFF-450_CTau-10000mm_TuneCP5_14TeV_pythia8.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4q_MH_125_MFF_1_CTau_10000mm_TuneCP5_14TeV_pythia.root',
             'trees/MaxDigis_tree_HTo2LongLivedTo4q_MH_125_MFF_1_CTau_5000mm_TuneCP5_14TeV_pythia.root',
            ]
comp_treename = 'comparator'
wire_treename = 'wire'

comp_var = [ 'Ev_max_nComp_ME11','Ev_max_nComp_ME12','Ev_max_nComp_ME13','Ev_max_nComp_ME21','Ev_max_nComp_ME22',
             'Ev_max_nComp_ME31','Ev_max_nComp_ME32','Ev_max_nComp_ME41','Ev_max_nComp_ME42']
wire_var = [ 'Ev_max_nWire_ME11','Ev_max_nWire_ME12','Ev_max_nWire_ME13','Ev_max_nWire_ME21','Ev_max_nWire_ME22',
             'Ev_max_nWire_ME31','Ev_max_nWire_ME32','Ev_max_nWire_ME41','Ev_max_nWire_ME42']

#all_vars = [ 'Ev_nComp_ME11','Ev_nComp_ME12']

comp_vars = list(set(comp_var))
wire_vars = list(set(wire_var))

arrs_comp_sig = [None]*(len(input_sig)+1)
arrs_wire_sig = [None]*(len(input_sig)+1)

print "Opening input files"

print "Opening file: ", first_sig[0]
arrs_comp_sig[0] = uproot.open(first_sig[0])[comp_treename]
arrs_wire_sig[0] = uproot.open(first_sig[0])[wire_treename]
comp_sig = arrs_comp_sig[0].pandas.df(comp_vars)
wire_sig = arrs_wire_sig[0].pandas.df(wire_vars)
if normalize:
    comp_sig = comp_sig.head(130)
    wire_sig = comp_sig.head(130)
print "Comp num events = %i" % (len(comp_sig.index))
print "Wire num events = %i" % (len(wire_sig.index))

j=1
for i in (input_sig):
    print "Opening file: ", i
    arrs_comp_sig[j] = uproot.open(i)[comp_treename]
    arrs_wire_sig[j] = uproot.open(i)[wire_treename]
    temp_comp_sig = arrs_comp_sig[j].pandas.df(comp_vars)
    temp_wire_sig = arrs_wire_sig[j].pandas.df(wire_vars)
    if normalize:
        temp_comp_sig = temp_comp_sig.head(130)
        temp_wire_sig = temp_wire_sig.head(130)
    print "Comp num events = %i" % (len(temp_comp_sig.index))
    print "Wire num events = %i" % (len(temp_wire_sig.index))
    comp_sig = comp_sig.append(temp_comp_sig)
    wire_sig = wire_sig.append(temp_wire_sig)
    j=j+1


print "Opening file: ", input_bkg[0]
arrs_comp_bkg = uproot.open(input_bkg[0])[comp_treename]
arrs_wire_bkg = uproot.open(input_bkg[0])[wire_treename]
comp_bkg = arrs_comp_bkg.pandas.df(comp_vars)
wire_bkg = arrs_wire_bkg.pandas.df(wire_vars)

Opening input files
Opening file:  trees/MaxDigis_tree_ppTohToSSTo4b.root
Comp num events = 8710
Wire num events = 8883
Opening file:  trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-25_CTau-15000mm_TuneCP5_14TeV_pythia8.root
Comp num events = 2525
Wire num events = 2528
Opening file:  trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-25_CTau-1500mm_TuneCP5_14TeV_pythia8.root
Comp num events = 6387
Wire num events = 6395
Opening file:  trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-12_CTau-9000mm_TuneCP5_14TeV_pythia8.root
Comp num events = 2535
Wire num events = 2537
Opening file:  trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-12_CTau-900mm_TuneCP5_14TeV_pythia8.root
Comp num events = 7282
Wire num events = 7297
Opening file:  trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-50_CTau-30000mm_TuneCP5_14TeV_pythia8.root
Comp num events = 3032
Wire num events = 3032
Opening file:  trees/MaxDigis_tree_HTo2LongLivedTo4b_MH-125_MFF-50_CTau-3000mm_TuneCP5_14TeV_pythia8.root
Comp num even

# Get Optimal Thresholds

In [3]:
min_=0
max_=150
it_=1

r1=2
r2=3
r = [r1,r2,r2,r1,r2,r1,r2,r1,r2]

l1=.07
l2=.28
rmin = [0.0,.02,.02,.02,.02,.02,.02,.02,.02]
rmax = [.02,l2,l2,l1,l2,l1,l2,l1,l2]


compXX = [0,0,0,0,0,0,0,0,0]
wireXX = [0,0,0,0,0,0,0,0,0]

#inc = 0

In [4]:
for inc in range(len(comp_var)):

    comp_efficiency=[0]
    comp_sig_tot = len(comp_sig)
    comp_rate=[0]
    comp_bkg_tot = len(comp_bkg)
    comp_limits=[0]
    best = 0.0
    best_cut = 149

    for limit in (xrange(min_,max_,it_)):
        comp_efficiency.append(len(comp_sig[(comp_sig[comp_var[inc]] > limit)])/comp_sig_tot*100)

        comp_rate.append(len(comp_bkg[(comp_bkg[comp_var[inc]] > limit)])/comp_bkg_tot*30*1000)

        comp_limits.append(limit)
        
    for i in range(len(comp_rate)):
        if comp_rate[i] > rmin[inc] and comp_rate[i] < rmax[inc]:
            if np.power(comp_efficiency[i],r[inc])/np.sqrt(comp_rate[i]) > best:
                best = np.power(comp_efficiency[i],r[inc])/np.sqrt(comp_rate[i])
                best_cut = comp_limits[i]

    print "For " + comp_var[inc] + ":"
    print "Best threshold > %i" %(best_cut)
    print 'rate =', comp_rate[best_cut+1], 'kHz and efficiency =', comp_efficiency[best_cut+1], '% for threshold >', comp_limits[best_cut+1]
    print "-----------------------------------------------------------------------"
    compXX[inc] = best_cut

efficiency_comp_final = len(comp_sig[(comp_sig['Ev_max_nComp_ME11'] > compXX[0]) | (comp_sig['Ev_max_nComp_ME12'] > compXX[1]) | (comp_sig['Ev_max_nComp_ME13'] > compXX[2]) |
                           (comp_sig['Ev_max_nComp_ME21'] > compXX[3]) | (comp_sig['Ev_max_nComp_ME22'] > compXX[4]) |
                           (comp_sig['Ev_max_nComp_ME31'] > compXX[5]) | (comp_sig['Ev_max_nComp_ME32'] > compXX[6]) |
                           (comp_sig['Ev_max_nComp_ME41'] > compXX[7]) | (comp_sig['Ev_max_nComp_ME41'] > compXX[8])])/comp_sig_tot*100
                
rate_comp_final       = len(comp_bkg[(comp_bkg['Ev_max_nComp_ME11'] > compXX[0]) | (comp_bkg['Ev_max_nComp_ME12'] > compXX[1]) | (comp_bkg['Ev_max_nComp_ME13'] > compXX[2]) |
                           (comp_bkg['Ev_max_nComp_ME21'] > compXX[3]) | (comp_bkg['Ev_max_nComp_ME22'] > compXX[4]) |
                           (comp_bkg['Ev_max_nComp_ME31'] > compXX[5]) | (comp_bkg['Ev_max_nComp_ME32'] > compXX[6]) |
                           (comp_bkg['Ev_max_nComp_ME41'] > compXX[7]) | (comp_bkg['Ev_max_nComp_ME42'] > compXX[8])])/comp_bkg_tot*30*1000

print "Combined Result:"
print "rate =", rate_comp_final, "kHz, efficiency =", efficiency_comp_final, "%"

For Ev_max_nComp_ME11:
Best threshold > 98
rate = 0.00966956205586 kHz and efficiency = 0.154841985023 % for threshold > 98
-----------------------------------------------------------------------
For Ev_max_nComp_ME12:
Best threshold > 54
rate = 0.270747737564 kHz and efficiency = 5.7913440792 % for threshold > 54
-----------------------------------------------------------------------
For Ev_max_nComp_ME13:
Best threshold > 21
rate = 0.232069489341 kHz and efficiency = 5.04886406904 % for threshold > 21
-----------------------------------------------------------------------
For Ev_max_nComp_ME21:
Best threshold > 49
rate = 0.0386782482235 kHz and efficiency = 5.44866099759 % for threshold > 49
-----------------------------------------------------------------------
For Ev_max_nComp_ME22:
Best threshold > 32
rate = 0.261078175508 kHz and efficiency = 10.1040741211 % for threshold > 32
-----------------------------------------------------------------------
For Ev_max_nComp_ME31:
Best thre

In [5]:
for inc in range(len(wire_var)):

    wire_efficiency=[0]
    wire_sig_tot = len(wire_sig)
    wire_rate=[0]
    wire_bkg_tot = len(wire_bkg)
    wire_limits=[0]
    best = 0.0
    best_cut = 149

    for limit in (xrange(min_,max_,it_)):
        wire_efficiency.append(len(wire_sig[(wire_sig[wire_var[inc]] > limit)])/wire_sig_tot*100)

        wire_rate.append(len(wire_bkg[(wire_bkg[wire_var[inc]] > limit)])/wire_bkg_tot*30*1000)

        wire_limits.append(limit)
        
    for i in range(len(wire_rate)):
        if wire_rate[i] > rmin[inc] and wire_rate[i] < rmax[inc]:
            if np.power(wire_efficiency[i],r[inc])/np.sqrt(wire_rate[i]) > best:
                best = np.power(wire_efficiency[i],r[inc])/np.sqrt(wire_rate[i])
                best_cut = wire_limits[i]

    print "For " + wire_var[inc] + ":"
    print "Best threshold > %i" %(best_cut)
    print 'rate =', wire_rate[best_cut+1], 'kHz and efficiency =', wire_efficiency[best_cut+1], '% for threshold >', wire_limits[best_cut+1]
    print "-----------------------------------------------------------------------"
    wireXX[inc] = best_cut

efficiency_wire_final = len(wire_sig[(wire_sig['Ev_max_nWire_ME11'] > wireXX[0]) | (wire_sig['Ev_max_nWire_ME12'] > wireXX[1]) | (wire_sig['Ev_max_nWire_ME13'] > wireXX[2]) |
                           (wire_sig['Ev_max_nWire_ME21'] > wireXX[3]) | (wire_sig['Ev_max_nWire_ME22'] > wireXX[4]) |
                           (wire_sig['Ev_max_nWire_ME31'] > wireXX[5]) | (wire_sig['Ev_max_nWire_ME32'] > wireXX[6]) |
                           (wire_sig['Ev_max_nWire_ME41'] > wireXX[7]) | (wire_sig['Ev_max_nWire_ME41'] > wireXX[8])])/wire_sig_tot*100
                
rate_wire_final       = len(wire_bkg[(wire_bkg['Ev_max_nWire_ME11'] > wireXX[0]) | (wire_bkg['Ev_max_nWire_ME12'] > wireXX[1]) | (wire_bkg['Ev_max_nWire_ME13'] > wireXX[2]) |
                           (wire_bkg['Ev_max_nWire_ME21'] > wireXX[3]) | (wire_bkg['Ev_max_nWire_ME22'] > wireXX[4]) |
                           (wire_bkg['Ev_max_nWire_ME31'] > wireXX[5]) | (wire_bkg['Ev_max_nWire_ME32'] > wireXX[6]) |
                           (wire_bkg['Ev_max_nWire_ME41'] > wireXX[7]) | (wire_bkg['Ev_max_nWire_ME42'] > wireXX[8])])/wire_bkg_tot*30*1000

print "Combined Result:"
print "rate =", rate_wire_final, "kHz, efficiency =", efficiency_wire_final, "%"

For Ev_max_nWire_ME11:
Best threshold > 104
rate = 0.0190141420851 kHz and efficiency = 0.352972432726 % for threshold > 104
-----------------------------------------------------------------------
For Ev_max_nWire_ME12:
Best threshold > 89
rate = 0.247183847106 kHz and efficiency = 4.79485849474 % for threshold > 89
-----------------------------------------------------------------------
For Ev_max_nWire_ME13:
Best threshold > 28
rate = 0.275705060234 kHz and efficiency = 4.81636577559 % for threshold > 28
-----------------------------------------------------------------------
For Ev_max_nWire_ME21:
Best threshold > 133
rate = 0.0285212131276 kHz and efficiency = 3.6296699265 % for threshold > 133
-----------------------------------------------------------------------
For Ev_max_nWire_ME22:
Best threshold > 83
rate = 0.0380282841702 kHz and efficiency = 2.78962083929 % for threshold > 83
-----------------------------------------------------------------------
For Ev_max_nWire_ME31:
Best 

# Results

In [6]:
print "Optimal Comparator Thresholds:"
print "ME11: %i, ME12: %i, ME13: %i," %(compXX[0],compXX[1],compXX[2])
print "ME21: %i, ME22: %i," %(compXX[3],compXX[4])
print "ME31: %i, ME32: %i," %(compXX[5],compXX[6])
print "ME41: %i, ME42: %i," %(compXX[7],compXX[8])
print "With rate and efficiency:"
print "rate = %f kHz, efficiency = %f%%" %(rate_comp_final,efficiency_comp_final)
print
print "Optimal Wire Thresholds:"
print "ME11: %i, ME12: %i, ME13: %i," %(wireXX[0],wireXX[1],wireXX[2])
print "ME21: %i, ME22: %i," %(wireXX[3],wireXX[4])
print "ME31: %i, ME32: %i," %(wireXX[5],wireXX[6])
print "ME41: %i, ME42: %i," %(wireXX[7],wireXX[8])
print "With rate and efficiency:"
print "rate = %f kHz, efficiency = %f%%" %(rate_wire_final,efficiency_wire_final)

Optimal Comparator Thresholds:
ME11: 98, ME12: 54, ME13: 21,
ME21: 49, ME22: 32,
ME31: 49, ME32: 35,
ME41: 48, ME42: 31,
With rate and efficiency:
rate = 0.986295 kHz, efficiency = 37.801751%

Optimal Wire Thresholds:
ME11: 104, ME12: 89, ME13: 28,
ME21: 133, ME22: 83,
ME31: 130, ME32: 74,
ME41: 127, ME42: 88,
With rate and efficiency:
rate = 0.893665 kHz, efficiency = 24.969953%
